<a href="https://colab.research.google.com/github/arnavjain2710/IITISoC24-ML-36/blob/main/Finetune_classification_emo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To load: if the files section isn't empty
First rename tess toronto---- ko underscore se seperate karke rename kardo
TESS Toronto wale ko bhi underscore se seperate karke rename karna hai after u run the first cell, jo upload hoga usme underscore nhi hoga because
create a folder called 'ravdess' in the content section

Firstly loading the datasets from kaggle, as these are too huge to be stored on computer

In [ ]:
! rm -rf ALL
! rm -rf AudioWAV
! rm -rf ravdess
! rm -rf audio_speech_actors_01-24
! rm -rf TESS_Toronto_emotional_speech_set_data
! rm -rf tess_toronto_emotional_speech_set_data
# name all folders as it has to be run before each iteration

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaidehibhat","key":"21ccd7dc13a7f52c2ecc8b29ff2e7dd3"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!mv kaggle.json ~/.kaggle/


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! pip install -q kaggle

In [ ]:
!kaggle datasets list

ref                                                        title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
kanchana1990/vehicle-dataset-2024                          Vehicle Dataset 2024                                308KB  2024-05-29 20:02:30            877         33  1.0              
jainaru/parkinson-disease-detection                        Parkinson Disease Detection                          16KB  2024-05-27 15:51:16            717         23  1.0              
piterfm/football-soccer-uefa-euro-1960-2024                Football - Soccer - UEFA EURO, 1960 - 2024           87KB  2024-05-27 19:28:57            765         23  1.0              
hamjashaikh/mental-health-detection-dataset                Mental Health Detection Da

Loading Crema-D from kaggle, already only audio files are there in the dataset

In [ ]:
! kaggle datasets download -d ejlok1/cremad

Dataset URL: https://www.kaggle.com/datasets/ejlok1/cremad
License(s): ODC Attribution License (ODC-By)
 98% 441M/451M [00:07<00:00, 40.4MB/s]
100% 451M/451M [00:07<00:00, 65.4MB/s]


Loading TESS dataset

In [ ]:
! kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess

Dataset URL: https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 422M/428M [00:03<00:00, 124MB/s]
100% 428M/428M [00:03<00:00, 114MB/s]


Loading SAVEE dataset, has only audio files in it

In [ ]:
! kaggle datasets download -d ejlok1/surrey-audiovisual-expressed-emotion-savee

Dataset URL: https://www.kaggle.com/datasets/ejlok1/surrey-audiovisual-expressed-emotion-savee
License(s): copyright-authors
 75% 81.0M/107M [00:01<00:00, 69.1MB/s]
100% 107M/107M [00:01<00:00, 88.5MB/s] 


Loading RAVDESS dataset, has only audio files in it

In [ ]:
! kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 99% 425M/429M [00:04<00:00, 97.8MB/s]
100% 429M/429M [00:04<00:00, 111MB/s] 


Unzipping the content

In [ ]:
! unzip /content/cremad.zip
! unzip /content/ravdess-emotional-speech-audio.zip
! unzip /content/surrey-audiovisual-expressed-emotion-savee.zip
! unzip /content/toronto-emotional-speech-set-tess.zip

Streaming output truncated to the last 5000 lines.
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_back_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bar_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_base_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bean_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_beg_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bite_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_boat_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bone_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_book_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bought_angry.wav  
  inflating: TESS Toronto emotional 

Making sense of data, all the datasets have these emotions in common:
1) Anger
2) Disgust
3) Happy
4) Neutral
5) Sad
6) Fear

Tip: remember that RAVDESS labels neutral as calm

Only Crema-D doesn't have surprise hence not including right now, later we can include that too, also when we get access to LSSED database we can start working with that

Work Plan: Create a double numpy array, with indexes as given above (-1 ofc) we will store the file paths in it.

In [ ]:
dataset_array = [[] for _ in range(6)]
#this will contain five arrays on indexes:  0: anger, 1:disgust, 2: Happy, 3: neutral/calm , 4:sad, 5: fear
#further each will contain subarrays containing file paths
#have to create directories of each dataset for it


In [ ]:
! pip install --upgrade librosa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.0
    Uninstalling librosa-0.8.0:
      Successfully uninstalled librosa-0.8.0


In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import librosa

In [ ]:
tess_path= '/content/TESS_Toronto_emotional_speech_set_data'
savee_path='/content/ALL'
cremad_path='/content/AudioWAV'
# have to work exrtra for ravdess


In [ ]:
for i in range(1,25):
  if i<10:
    file_path= f'/content/Actor_0{i}'
    print(file_path)
    !mv {file_path} /content/ravdess


  else:
    file_path= f'/content/Actor_{i}'
    print(file_path)
    !mv {file_path} /content/ravdess





/content/Actor_01
/content/Actor_02
/content/Actor_03
/content/Actor_04
/content/Actor_05
/content/Actor_06
/content/Actor_07
/content/Actor_08
/content/Actor_09
/content/Actor_10
/content/Actor_11
/content/Actor_12
/content/Actor_13
/content/Actor_14
/content/Actor_15
/content/Actor_16
/content/Actor_17
/content/Actor_18
/content/Actor_19
/content/Actor_20
/content/Actor_21
/content/Actor_22
/content/Actor_23
/content/Actor_24


In [ ]:
ravdess_path='/content/ravdess'

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt


In [ ]:
file_emo_path='/content/dataset.csv'
file_emo=pd.read_csv(file_emo_path)
final_emo_dataset=[]

labels = file_emo['label'].unique()
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

for index, row in file_emo.iterrows():
    # Create the dictionary for each row
    audio_array, sr = librosa.load(row['file_path'], sr=16000)
    entry = {

        'audio': {
            'array': audio_array,
            'path': row['file_path'],
            'sampling_rate': sr
        },
        'label': int(label2id[str(row['label'])])  # Convert intent_class to string
    }

    # Add the entry to the dataset list
    final_emo_dataset.append(entry)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
print(final_emo_dataset[0])

{'audio': {'array': array([0.00167847, 0.00164795, 0.00305176, ..., 0.        , 0.        ,
       0.        ], dtype=float32), 'path': '/content/AudioWAV/1043_TAI_NEU_XX.wav', 'sampling_rate': 16000}, 'label': 0}


In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
!pip install datasets

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
from datasets import Dataset, concatenate_datasets


chunk_size = 100

dataset_chunks = []

for i in range(0, len(final_emo_dataset), chunk_size):

    chunk_data = final_emo_dataset[i:i + chunk_size]


    chunk = Dataset.from_dict({key: [example[key] for example in chunk_data] for key in final_emo_dataset[0]})
    dataset_chunks.append(chunk)

dataset = concatenate_datasets(dataset_chunks)


In [ ]:
dataset=dataset.train_test_split(test_size=0.2)

In [ ]:
print(dataset["train"][0])

{'audio': {'array': [0.0015869140625, 0.00152587890625, 0.001953125, 0.002105712890625, 0.0018310546875, 0.002777099609375, 0.002197265625, 0.0020751953125, 0.00225830078125, 0.001678466796875, 0.000946044921875, 0.0018310546875, 0.00213623046875, 0.002166748046875, 0.002349853515625, 0.0018310546875, 0.001373291015625, 0.000640869140625, 0.000579833984375, 0.00048828125, 0.000152587890625, -0.000946044921875, -0.0009765625, -0.001617431640625, -0.0018310546875, -0.00299072265625, -0.00335693359375, -0.003936767578125, -0.003631591796875, -0.0037841796875, -0.004425048828125, -0.00506591796875, -0.00531005859375, -0.003509521484375, -0.003143310546875, -0.003631591796875, -0.004119873046875, -0.003631591796875, -0.00323486328125, -0.003173828125, -0.002960205078125, -0.002716064453125, -0.00225830078125, -0.002197265625, -0.0018310546875, -0.00140380859375, -0.000274658203125, -0.00030517578125, -0.000274658203125, 9.1552734375e-05, 6.103515625e-05, -3.0517578125e-05, 0.000701904296875

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [ ]:
dataset= dataset.map(preprocess_function,  batched=True)


Map:   0%|          | 0/9096 [00:00<?, ? examples/s]

Map:   0%|          | 0/2274 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install accelerate -U

In [ ]:
print(type(dataset))
print(type(dataset["train"]))

<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
training_args = TrainingArguments(
    output_dir="fine_tuned_emo_classif",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss,Accuracy
0,1.597000,1.566588,0.367634
1,1.314900,1.281748,0.537379
2,1.209200,1.145252,0.585752
4,1.031900,1.049388,0.620493
5,0.943000,1.023260,0.620053
6,0.891900,0.968788,0.642480
8,0.889900,0.950598,0.648637
9,0.827700,0.955976,0.644239


TrainOutput(global_step=710, training_loss=1.103531505692173, metrics={'train_runtime': 3409.0399, 'train_samples_per_second': 26.682, 'train_steps_per_second': 0.208, 'total_flos': 8.23114097121792e+17, 'train_loss': 1.103531505692173, 'epoch': 9.964912280701755})

In [ ]:
!pip install huggingface-cli


In [ ]:
!pip install huggingface-hub
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
trainer.push_to_hub("VaidehiBhat007/Finetune_classification_emo")

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

events.out.tfevents.1718019736.02f02e688f91.3426.0:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/VaidehiBhat007/fine_tuned_emo_classif/commit/48adc5eb2ef79c9b7e7ac70a86835c620d46d15a', commit_message='VaidehiBhat007/Finetune_classification_emo', commit_description='', oid='48adc5eb2ef79c9b7e7ac70a86835c620d46d15a', pr_url=None, pr_revision=None, pr_num=None)